In [ ]:
!pip uninstall -y pyarrow
!pip install ray==1.10.0

     |████████████████████████████████| 59.6 MB 90 kB/s 
  Attempting uninstall: ray
    Found existing installation: ray 0.7.5
    Uninstalling ray-0.7.5:
      Successfully uninstalled ray-0.7.5


상태가 존재하는 actor와 그들의 메서드를 배우는 작업
(remote 함수는 stateless)



In [ ]:
import ray

ray.init()

{'gcs_address': '172.28.0.2:44497',
 'metrics_export_port': 63242,
 'node_id': '9eb25319c7a722b5485f9507053485585f9e13e7007e171c1b90da56',
 'node_ip_address': '172.28.0.2',
 'object_store_address': '/tmp/ray/session_2022-02-11_13-20-13_882213_928/sockets/plasma_store',
 'raylet_ip_address': '172.28.0.2',
 'raylet_socket_name': '/tmp/ray/session_2022-02-11_13-20-13_882213_928/sockets/raylet',
 'redis_address': '172.28.0.2:6379',
 'session_dir': '/tmp/ray/session_2022-02-11_13-20-13_882213_928',
 'webui_url': None}

In [ ]:
@ray.remote
class Example(object):
    def __init__(self, x):
        self.x = x

    def set(self, x):
        self.x = x

    def get(self):
        return self.x

e = Example.remote(1)
print(e.set.remote(2))
print(e.get.remote())

print(ray.get(e.set.remote(2)))
print(ray.get(e.get.remote()))

ObjectRef(63964fa4841d4a2ec30d1fe593dc0533761387200100000001000000)
ObjectRef(69a6825d641b4613c30d1fe593dc0533761387200100000001000000)
None
2


In [ ]:
import time

@ray.remote
class Foo(object):
    def __init__(self):
        self.counter = 0
    
    def reset(self):
        self.counter = 0

    def increment(self):
        time.sleep(0.5)
        self.counter += 1
        return self.counter

In [ ]:
f1 = Foo.remote()
f2 = Foo.remote()
f3 = Foo.remote()

In [ ]:
start_time = time.time()

f1.reset.remote()
f2.reset.remote()
f3.reset.remote()

results = []
for _ in range(10):
    results.append(f1.increment.remote())
    results.append(f2.increment.remote())
    results.append(f3.increment.remote())

results = ray.get(results)

duration = time.time() - start_time

print(duration)

5.02685546875


actor간 내용을 공유할 수 있도록 함

In [ ]:
@ray.remote
class Actor(object):
    def method(self):
        pass

actor = Actor.remote()

@ray.remote
def f(actor):
    return ray.get(actor.method.remote())

f.remote(actor)
f.remote(actor)
f.remote(actor)

ObjectRef(30c04f84db70b40cffffffffffffffffffffffff0100000001000000)

In [ ]:
from collections import defaultdict

@ray.remote
class LoggingActor(object):
    def __init__(self):
        self.logs = defaultdict(lambda: [])

    def log(self, index, message):
        self.logs[index].append(message)

    def get_logs(self):
        return dict(self.logs)

logging_actor = LoggingActor.remote()

In [ ]:
@ray.remote
def run_experiment(experiment_index, logging_actor):
    for i in range(60):
        time.sleep(1)
        logging_actor.log.remote(experiment_index, f"on iteration {i}")

In [ ]:
experiment_ids = []
for i in range(3):
    experiment_ids.append(run_experiment.remote(i, logging_actor))

In [ ]:
logs = ray.get(logging_actor.get_logs.remote())

In [ ]:
logs

{0: ['on iteration 0',
  'on iteration 1',
  'on iteration 2',
  'on iteration 3',
  'on iteration 4',
  'on iteration 5',
  'on iteration 6',
  'on iteration 7',
  'on iteration 8',
  'on iteration 9',
  'on iteration 10',
  'on iteration 11',
  'on iteration 12',
  'on iteration 13',
  'on iteration 14',
  'on iteration 15',
  'on iteration 16',
  'on iteration 17'],
 1: ['on iteration 0',
  'on iteration 1',
  'on iteration 2',
  'on iteration 3',
  'on iteration 4',
  'on iteration 5',
  'on iteration 6',
  'on iteration 7',
  'on iteration 8',
  'on iteration 9',
  'on iteration 10',
  'on iteration 11',
  'on iteration 12',
  'on iteration 13',
  'on iteration 14',
  'on iteration 15',
  'on iteration 16',
  'on iteration 17']}